In [1]:
from tensorflow.keras.datasets import cifar10
import numpy as np
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.applications.vgg19 import VGG19, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
import cv2
from keras_preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import backend as BE
from tensorflow.keras.losses import categorical_crossentropy
from progressbar import ProgressBar
import os
import scipy.io
import tensorflow as tf
import json
import sys
sys.path.append('..')  
from GradPri_utils.utils import *

# from tensorflow.keras import backend as BE
# from Integrated_Gradients_algorithm import *
# from GradVisualizer import *

os.environ["CUDA_VISIBLE_DEVICES"] = "4"
config=tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess=tf.compat.v1.Session(config=config) 

# tf.compat.v1.disable_eager_execution()
# tf.compat.v1.enable_eager_execution()

In [2]:
base_model = VGG19(weights='imagenet')

In [3]:
# data = np.load('./featureExtraction/ImageNet_1616samples_AdvOrg.npy', allow_pickle=True)
data = np.load('./featureExtraction/ImageNet_556samples_OneStepAdvOrg.npy', allow_pickle=True)

In [4]:
data.item().keys()

dict_keys(['X_adv', 'X_org', 'ground_truth_label', 'X_adv_predicted_confidence', 'predicted_confidence', 'index1'])

In [16]:
top_set = 1 # 预测的前n个类中包含真实标签则表示预测正确
total_sample_num = len(data.item()['index1'])
img_num = range(0, total_sample_num)
ground_truth_label = []  # 记录样本的真实标签
predicted_confidence = []  # 记录样本的置信度信息
binary_flag = [] # 记录是否为对抗样本，是为1，否为0，误分类的为-1
gradients = []
second_gradients = []
num_class = 1000

# 先计算对抗样本的梯度
pbar = ProgressBar()
for i in pbar(img_num):
    x_tmp = data.item()['X_adv'][i]
#     pre_tmp = base_model.predict(np.reshape(x_tmp, [-1,224,224,3]))
    pre_tmp = data.item()['X_adv_predicted_confidence'][i]
    predicted_confidence.append(pre_tmp)
    label_tmp = data.item()['ground_truth_label'][i]
    ground_truth_label.append(label_tmp)
    binary_flag.append(1)
    
#     target_one_hot = np.reshape(to_categorical(np.argmax(pre_tmp), num_class),(1,-1))
#     grads = get_loss_gradients(x_tmp, base_model, target_one_hot)
    grads = PreGradientEstimator(samples = 6, sigma=1, model = base_model, x=x_tmp, 
                                 bounds=(np.min(x_tmp),np.max(x_tmp)), noise_mu=0, nise_std=1, 
                                 top_pred_idx = np.argmax(pre_tmp), clip=True)
    gradients.append(grads)
    
#     grads = get_gradients(x_tmp, base_model, np.argmax(pre_tmp))
#     gradients.append(grads)
#     target_label = np.argsort(-pre_tmp) 
#     target_label = target_label[:,1][0]
#     grads = get_gradients(x_tmp, base_model, target_label)
#     second_gradients.append(grads)
    
# 再计算正常样本的梯度
pbar = ProgressBar()
for i in pbar(img_num):
    x_tmp = data.item()['X_org'][i]
#     pre_tmp = base_model.predict(np.reshape(x_tmp, [-1,224,224,3]))
    pre_tmp = data.item()['predicted_confidence'][i]
    predicted_confidence.append(pre_tmp)
    label_tmp = data.item()['ground_truth_label'][i]
    ground_truth_label.append(label_tmp)
    binary_flag.append(0)
    
#     target_one_hot = np.reshape(to_categorical(np.argmax(pre_tmp), num_class),(1,-1))
#     grads = get_loss_gradients(x_tmp, base_model, target_one_hot)
    grads = PreGradientEstimator(samples = 6, sigma=1, model = base_model, x=x_tmp, 
                                 bounds=(np.min(x_tmp),np.max(x_tmp)), noise_mu=0, nise_std=1,
                                 top_pred_idx = np.argmax(pre_tmp), clip=True)
    gradients.append(grads)
    
#     grads = get_gradients(x_tmp, base_model, np.argmax(pre_tmp))
#     gradients.append(grads)
#     target_label = np.argsort(-pre_tmp) 
#     target_label = target_label[:,1][0]
#     grads = get_gradients(x_tmp, base_model, target_label)
#     second_gradients.append(grads)

# # 最后计算原始数据集中被误分类的样本的梯度
# DATA_PATH = '../datasets/ImageNetVal/'
# file_name = getfile_name(DATA_PATH)
# file_name = np.sort(file_name) 
# synsets = scipy.io.loadmat(os.path.join('../datasets', 'ILSVRC2012_devkit_t12', 'data', 'meta.mat'))['synsets']
# WNID = [s[0][1][0] for s in synsets]
# f = open("../datasets/ILSVRC2012_devkit_t12/data/ILSVRC2012_validation_ground_truth.txt",encoding = "utf-8")
# val_ground_truth = f.read()
# val_ground_truth = val_ground_truth.split('\n')
# for i in range(len(val_ground_truth)-1):
#     val_ground_truth[i] = int(val_ground_truth[i])

# pbar = ProgressBar()
# for i in pbar(range(0,3000)):
#     img_path = DATA_PATH + file_name[i]
#     img = image.load_img(img_path, target_size=(224, 224))
#     x_tmp = image.img_to_array(img)
#     x_tmp = np.expand_dims(x_tmp, axis=0)
#     x_tmp = preprocess_input(x_tmp)
#     pre_tmp = base_model.predict(np.reshape(x_tmp, [-1,224,224,3]))
    
#     if not get_acc(predict_label=decode_predictions(pre_tmp, top=top_set), 
#                ground_truth=WNID[val_ground_truth[i]-1]): # 选取被错误分类的样本
#         pre_tmp = base_model.predict(np.reshape(x_tmp, [-1,224,224,3]))
#         predicted_confidence.append(pre_tmp)
#         ground_truth_label.append(WNID[val_ground_truth[i]-1])

#         target_one_hot = np.reshape(to_categorical(np.argmax(pre_tmp), num_class),(1,-1))
#         grads = get_loss_gradients(x_tmp, base_model, target_one_hot)
#         gradients.append(grads)
#         binary_flag.append(-1)

100% |########################################################################|
100% |########################################################################|


In [17]:
np.shape(binary_flag)

(1112,)

In [69]:
# np.save('./featureExtraction/ImageNet_'+str(total_sample_num)+'samples_AdvOrgMis_GradientFeature.npy',{
#     'ground_truth_label': np.array(ground_truth_label),
#     'predicted_confidence': np.array(predicted_confidence),
#     'binary_flag': np.array(binary_flag),
#     'gradients': np.array(gradients),
# })

In [18]:
Gini = []
pbar = ProgressBar()
for i in pbar(range(0, len(binary_flag))):
    Gini.append(np.sum(np.abs(gradients[i])))
    
#     tmp1 = (np.abs(second_gradients[i]) - np.abs(gradients[i]))/np.abs(gradients[i])
#     Gini.append(np.sum(np.abs(tmp1)))
    
#     Gini.append(1-np.sum(predicted_confidence[i]*predicted_confidence[i]))

100% |########################################################################|


In [20]:
indexs = np.argsort(Gini)
# indexs = indexs[::-1]
# 计算APFD指标
APFD,_,wrong_num_index = get_APFD(Gini_indexs=indexs, ground_truth_label=np.array(ground_truth_label), 
                    predicted_confidence=np.array(predicted_confidence), top_set=top_set,
                   decode_predictions=decode_predictions)
print("APFD: ", APFD)

# 计算RAUC指标
RAUC,_,_ = get_RAUC(Gini_indexs=indexs, ground_truth_label=np.array(ground_truth_label), 
                    predicted_confidence=np.array(predicted_confidence), top_set=top_set,
                    decode_predictions=decode_predictions)
print("RAUC: ", RAUC)

100% |########################################################################|


APFD:  0.639857538429688
RAUC:  0.8520330530063667


In [101]:
np.shape(tmp1)

(1, 224, 224, 3)